In [1]:
using LCIO
using Plots
using Glob
using LsqFit
gr()

Plots.GRBackend()

In [2]:
# read the files for calibration
fileList = readdir(glob"K0L*barrel*", "SingleParticles")

# build the list of input energies
energyString = r"_(\d+)GeV"
# get the energy string from the filename, convert from string to int, build a dictionary from ints to filenames
energyMap = Dict((parse(Int16, match(energyString, fn)[1]), fn) for fn in fileList)
println(energyMap)

Dict(20=>"SingleParticles/K0L_20GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio",100=>"SingleParticles/K0L_100GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio",10=>"SingleParticles/K0L_10GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio",50=>"SingleParticles/K0L_50GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio",2=>"SingleParticles/K0L_2GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio",5=>"SingleParticles/K0L_5GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio",1=>"SingleParticles/K0L_1GeV_theta5-175_barrel_sidloi3_scint3x3_pandoraREC.slcio")


In [3]:
function getHitsFromFile(filename)
    eCalEnergies = Float64[]
    hCalEnergies = Float64[]
    eCalLengths = Int16[]
    hCalLengths = Int16[]
    LCIO.open(filename) do reader
        for (idx, event) in enumerate(reader)
            if idx > 10000
                break
            end
            hCalEnergy = 0.0
            hCalLength = 0
            eCalEnergy = 0.0
            eCalLength = 0
            # for cn in getCollectionNames(event)
            #     println(cn)
            # end
            # break
            # sum up the uncalibrated HCalHits
            for hit in getCollection(event, "HcalBarrelHits")
                hCalEnergy += getEnergy(hit)
                hCalLength += 1
            end
            # sum up the calibrated ECalHits
            for hit in getCollection(event, "EM_BARREL")
                eCalEnergy += getEnergy(hit)
                eCalLength += 1
            end
            push!(eCalEnergies, eCalEnergy)
            push!(eCalLengths, eCalLength)
            push!(hCalEnergies, hCalEnergy)
            push!(hCalLengths, hCalLength)
        end
    end
    return eCalEnergies, eCalLengths, hCalEnergies, hCalLengths
end

getHitsFromFile (generic function with 1 method)

In [ ]:
eHits  = Dict{Int16, Vector{Float64}}()
eCount = Dict{Int16, Vector{Int16}}()
hHits  = Dict{Int16, Vector{Float64}}()
hCount = Dict{Int16, Vector{Int16}}()
for (energy, filename) in energyMap
    eCal, nEhits, hCal, nHhits = getHitsFromFile(filename);
    eHits[energy] = eCal
    eCount[energy] = nEhits
    hHits[energy] = hCal
    hCount[energy] = nHhits
end

In [ ]:
histogram([eHits[5] .+ hHits[5], eHits[10] .+ hHits[10], eHits[20].+ hHits[20]])

In [ ]:
# for now, we'll only adjust the hcal calibration
# pass the data in as x[1]=ecal x[2]=hcal
function fitter(ecal, hcal, energy)
    function model(x, p)
        return ecal + p[1] .* hcal
    end
    fit = curve_fit(model, hcal, energy, [0.5])
    return fit.param
end
# let's now assume x is the dictionary of energies and hits
function linearityModel(x, p)
    
end
for energy in keys(hHits)
    ecal = eHits[energy]
    hcal = hHits[energy]
    println(energy, '\t', fitter(ecal, hcal, energy))
end

In [ ]:
histogram(eCal .+ α .*hCal)

In [ ]:
histogram(nEhits)

In [ ]:
histogram(nHhits)

In [ ]:
histogram(nEhits+nHhits)

In [ ]:
x = nEhits+nHhits

In [ ]:
x[x.<30]